In [1]:
'''
http://2018.igem.org/wiki/images/0/09/2018_InterLab_Plate_Reader_Protocol.pdf
'''
import paml
import uml
import sbol3
from tyto import OM
from paml.execution_engine import ExecutionEngine
from paml_convert.markdown.markdown_specialization import MarkdownSpecialization


doc = sbol3.Document()
sbol3.set_namespace('http://igem.org/engineering/')

#############################################
# Import the primitive libraries
print('Importing libraries')
paml.import_library('liquid_handling')
print('... Imported liquid handling')
paml.import_library('plate_handling')
# print('... Imported plate handling')
paml.import_library('spectrophotometry')
print('... Imported spectrophotometry')
paml.import_library('sample_arrays')
print('... Imported sample arrays')
paml.import_library('culturing')
#############################################


# create the materials to be provisioned
dh5alpha = sbol3.Component('dh5alpha', 'https://identifiers.org/pubchem.substance:24901740')
dh5alpha.name = 'E. coli DH5 alpha'  
doc.add(dh5alpha)

lb_cam = sbol3.Component('lb_cam', 'https://identifiers.org/pubchem.substance:24901740')
lb_cam.name = 'LB Broth + chloramphenicol'  
doc.add(lb_cam)

chloramphenicol = sbol3.Component('chloramphenicol', 'https://identifiers.org/pubchem.substance:24901740')
chloramphenicol.name = 'chloramphenicol'  
doc.add(chloramphenicol)


neg_control_plasmid = sbol3.Component('neg_control_plasmid', sbol3.SBO_DNA)
neg_control_plasmid.name = 'Negative control'
neg_control_plasmid.description = 'BBa_R0040 Kit Plate 7 Well 2D'

pos_control_plasmid = sbol3.Component('pos_control_plasmid', sbol3.SBO_DNA)
pos_control_plasmid.name = 'Positive control'
pos_control_plasmid.description = 'BBa_I20270 Kit Plate 7 Well 2B'

test_device1 = sbol3.Component('test_device1', sbol3.SBO_DNA)
test_device1.name = 'Test Device 1'
test_device1.description = 'BBa_J364000 Kit Plate 7 Well 2F'

test_device2 = sbol3.Component('test_device2', sbol3.SBO_DNA)
test_device2.name = 'Test Device 2'
test_device2.description = 'BBa_J364001 Kit Plate 7 Well 2H'

test_device3 = sbol3.Component('test_device3', sbol3.SBO_DNA)
test_device3.name = 'Test Device 3'
test_device3.description = 'BBa_J364002 Kit Plate 7 Well 2J'

test_device4 = sbol3.Component('test_device4', sbol3.SBO_DNA)
test_device4.name = 'Test Device 4'
test_device4.description = 'BBa_J364007 Kit Plate 7 Well 2L'

test_device5 = sbol3.Component('test_device5', sbol3.SBO_DNA)
test_device5.name = 'Test Device 5'
test_device5.description = 'BBa_J364008 Kit Plate 7 Well 2N'

test_device6 = sbol3.Component('test_device6', sbol3.SBO_DNA)
test_device6.name = 'Test Device 6'
test_device6.description = 'BBa_J364009 Kit Plate 7 Well 2P'

doc.add(neg_control_plasmid)
doc.add(pos_control_plasmid)
doc.add(test_device1)
doc.add(test_device2)
doc.add(test_device3)
doc.add(test_device4)
doc.add(test_device5)
doc.add(test_device6)


protocol = paml.Protocol('interlab')
protocol.name = 'Cell measurement protocol'
protocol.description = '''Prior to performing the cell measurements you should perform all three of
the calibration measurements. Please do not proceed unless you have
completed the three calibration protocols.
Completion of the calibrations will ensure that you understand the measurement process and that
you can take the cell measurements under the same conditions. For the sake of consistency and
reproducibility, we are requiring all teams to use E. coli K-12 DH5-alpha. If you do not have access to
this strain, you can request streaks of the transformed devices from another team near you, and this
can count as a collaboration as long as it is appropriately documented on both teams' wikis. If you
are absolutely unable to obtain the DH5-alpha strain, you may still participate in the InterLab study
by contacting the Measurement Committee (measurement at igem dot org) to discuss your
situation.
For all of these cell measurements, you must use the same plates and volumes that you used in your
calibration protocol. You must also use the same settings (e.g., filters or excitation and emission
wavelengths) that you used in your calibration measurements. If you do not use the same plates,
volumes, and settings, the measurements will not be valid.'''
doc.add(protocol)




Importing libraries
... Imported liquid handling
... Imported spectrophotometry
... Imported sample arrays


In [2]:
plasmids = [neg_control_plasmid, pos_control_plasmid, test_device1, test_device2, test_device3, test_device4, test_device5, test_device6]

# Day 1: Transformation
transformation = protocol.primitive_step(f'Transform',
                                          host=dh5alpha,
                                          dna=plasmids,
                                          selection_medium=lb_cam)
    
# Day 2: Pick colonies and culture overnight
culture_container_day1 = protocol.primitive_step('EmptyContainer', 
                                                 specification=paml.ContainerSpec(name=f'culture (day 1)',
                                                                                  queryString='cont:CultureTube', 
                                                                                  prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))
            
overnight_culture = protocol.primitive_step('Culture',
                                            inoculum=transformation.output_pin('transformants'),
                                            growth_medium=lb_cam,
                                            volume=sbol3.Measure(5, OM.millilitre),  # Actually 5-10 ml in the written protocol
                                            duration=sbol3.Measure(16, OM.hour), # Actually 16-18 hours
                                            orbital_shake_speed=sbol3.Measure(220, None),  # No unit for RPM or inverse minutes
                                            temperature=sbol3.Measure(37, OM.degree_Celsius),
                                            container=culture_container_day1.output_pin('samples'))

# Execute the protocol

from IPython.display import Markdown

agent = sbol3.Agent("test_agent")
ee = ExecutionEngine(specializations=[MarkdownSpecialization("test_LUDOX_markdown.md")])
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=[])
Markdown(ee.specializations[0].markdown)

# Cell measurement protocol

## Description:
Prior to performing the cell measurements you should perform all three of
the calibration measurements. Please do not proceed unless you have
completed the three calibration protocols.
Completion of the calibrations will ensure that you understand the measurement process and that
you can take the cell measurements under the same conditions. For the sake of consistency and
reproducibility, we are requiring all teams to use E. coli K-12 DH5-alpha. If you do not have access to
this strain, you can request streaks of the transformed devices from another team near you, and this
can count as a collaboration as long as it is appropriately documented on both teams' wikis. If you
are absolutely unable to obtain the DH5-alpha strain, you may still participate in the InterLab study
by contacting the Measurement Committee (measurement at igem dot org) to discuss your
situation.
For all of these cell measurements, you must use the same plates and volumes that you used in your
calibration protocol. You must also use the same settings (e.g., filters or excitation and emission
wavelengths) that you used in your calibration measurements. If you do not use the same plates,
volumes, and settings, the measurements will not be valid.


## Protocol Materials:
* [E. coli DH5 alpha](https://identifiers.org/pubchem.substance:24901740)
* [LB Broth + chloramphenicol](https://identifiers.org/pubchem.substance:24901740)
* [chloramphenicol](https://identifiers.org/pubchem.substance:24901740)
* [Negative control](https://identifiers.org/SBO:0000251)
* [Positive control](https://identifiers.org/SBO:0000251)
* [Test Device 1](https://identifiers.org/SBO:0000251)
* [Test Device 2](https://identifiers.org/SBO:0000251)
* [Test Device 3](https://identifiers.org/SBO:0000251)
* [Test Device 4](https://identifiers.org/SBO:0000251)
* [Test Device 5](https://identifiers.org/SBO:0000251)
* [Test Device 6](https://identifiers.org/SBO:0000251)


## Protocol Inputs:


## Protocol Outputs:


## Steps
1. Transform `Negative control` DNA into `E. coli DH5 alpha` and plate transformants on LB Broth + chloramphenicol.Repeat for the remaining DNA samples: `Positive control`, `Test Device 1`, `Test Device 2`, `Test Device 3`, `Test Device 4`, `Test Device 5`, `Test Device 6`.
2. Provision a culture tube to contain `culture (day 1)`
3. Inoculate `E. coli DH5 alpha + Negative control transformants` into 5.0 milliliter of LB Broth + chloramphenicol in culture (day 1) and grow for 16.0 hour at 37.0 degree Celsius and 220.0 rpm. Repeat this procedure for the other inocula: `E. coli DH5 alpha + Positive control transformants`, `E. coli DH5 alpha + Test Device 1 transformants`, `E. coli DH5 alpha + Test Device 2 transformants`, `E. coli DH5 alpha + Test Device 3 transformants`, `E. coli DH5 alpha + Test Device 4 transformants`, `E. coli DH5 alpha + Test Device 5 transformants`, `E. coli DH5 alpha + Test Device 6 transformants`.
4. Report values for .


In [ ]:



timepoint_samples_0hrs = []  # Collection of samples on which we will run abs and fluorescence measurements
timepoint_samples_6hrs = []

plasmids = [neg_control_plasmid, pos_control_plasmid, test_device1, test_device2, test_device3, test_device4, test_device5, test_device6]
transformant_colonies = []
i_transformant_clone = 0
for plasmid in plasmids[:2]:
    
    # Day 1: Transformation
    transformation = protocol.primitive_step(f'Transform',
                                             host=dh5alpha,
                                             dna=plasmid,
                                             selection_medium=lb_cam)
    
    # Day 2: Pick colonies and culture overnight
    for i_replicates in range(0, 1):

        i_transformant_clone += 1
        culture_container_day1 = protocol.primitive_step('EmptyContainer', 
                                                     specification=paml.ContainerSpec(name=f'{plasmid.name} culture (day 1)',
                                                                                      queryString='cont:CultureTube', 
                                                                                      prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))
            
        overnight_culture = protocol.primitive_step('Culture',
                                               inoculum=transformation.output_pin('transformants'),
                                               growth_medium=lb_cam,
                                               volume=sbol3.Measure(5, OM.millilitre),  # Actually 5-10 ml in the written protocol
                                               duration=sbol3.Measure(16, OM.hour), # Actually 16-18 hours
                                               orbital_shake_speed=sbol3.Measure(220, None),  # No unit for RPM or inverse minutes
                                               temperature=sbol3.Measure(37, OM.degree_Celsius),
                                               container=culture_container_day1.output_pin('samples'))

        # Day 3 culture
        culture_container_day2 = protocol.primitive_step('EmptyContainer', 
                                                     specification=paml.ContainerSpec(name=f'{plasmid.name} culture (day 2)',
                                                                                      queryString='cont:CultureTube', 
                                                                                      prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))
        
        back_dilution = protocol.primitive_step('Dilute',
                                                source=culture_container_day1.output_pin('samples'),
                                                destination=culture_container_day2.output_pin('samples'),
                                                diluent=lb_cam,
                                                amount=sbol3.Measure(5.0, OM.millilitre),
                                                dilution_factor=uml.LiteralInteger(value=10))

        absorbance = protocol.primitive_step('MeasureAbsorbance',
                                             samples=culture_container_day2.output_pin('samples'),
                                             wavelength=sbol3.Measure(600, OM.nanometer))

        conical_tube = protocol.primitive_step('EmptyContainer', 
                                               specification=paml.ContainerSpec(name=f'{plasmid.name} culture',
                                               queryString='cont:ConicalTube', 
                                               prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))  # Should be opaque

        dilution = protocol.primitive_step('DiluteToTargetOD',
                                           source=culture_container_day2.output_pin('samples'),
                                           destination=conical_tube.output_pin('samples'),
                                           diluent=lb_cam,
                                           amount=sbol3.Measure(12, OM.millilitre),
                                           target_od=sbol3.Measure(0.2, None))  # Dilute to a target OD of 0.2, opaque container
        
        microfuge_tube_0hrs = protocol.primitive_step('EmptyContainer', 
                                                      specification=paml.ContainerSpec(name=f'{plasmid.name} culture (0 hrs)',
                                                      queryString='cont:MicrofugeTube', 
                                                      prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))

        transfer = protocol.primitive_step('Transfer',
                                           source=conical_tube.output_pin('samples'),
                                           destination=microfuge_tube_0hrs.output_pin('samples'),
                                           amount=sbol3.Measure(0.5, OM.milliliter))

        hold = protocol.primitive_step('Hold',
                                       location=microfuge_tube_0hrs.output_pin('samples'),
                                       temperature=sbol3.Measure(4, OM.degree_Celsius))
        
        incubate = protocol.primitive_step('Incubate',
                                           location=conical_tube.output_pin('samples'),
                                           duration=sbol3.Measure(6, OM.hour),
                                           temperature=sbol3.Measure(37, OM.degree_Celsius),
                                           shakingFrequency=sbol3.Measure(220, None))
        
        microfuge_tube_6hrs = protocol.primitive_step('EmptyContainer', 
                                                      specification=paml.ContainerSpec(name=f'{plasmid.name} culture (6 hrs)',
                                                      queryString='cont:MicrofugeTube', 
                                                      prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))
        
        
        transfer = protocol.primitive_step('Transfer',
                                           source=conical_tube.output_pin('samples'),
                                           destination=microfuge_tube_6hrs.output_pin('samples'),
                                           amount=sbol3.Measure(0.5, OM.milliliter))        

        hold = protocol.primitive_step('Hold',
                                       location=microfuge_tube_6hrs.output_pin('samples'),
                                       temperature=sbol3.Measure(4, OM.degree_Celsius))
        
        timepoint_samples_0hrs.append(microfuge_tube_0hrs)
        timepoint_samples_6hrs.append(microfuge_tube_6hrs)


        
# Transfer culture samples to a microwell plate for absorbance/fluorescence measurements        
plate = protocol.primitive_step('EmptyContainer', 
                                specification=paml.ContainerSpec(name=f'measurement plate',
                                queryString='cont:Plate96Well', 
                                prefixMap={'cont': 'https://sift.net/container-ontology/container-ontology#'}))

# Plate 0hr timepoint samples
for i_sample, sample in enumerate(timepoint_samples_0hrs):
        
    # Plate one sample per column, replicates in rows A-D
    column = i_sample + 1
    replicate1 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'A{column}')
    replicate2 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'B{column}')
    replicate3 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'C{column}')
    replicate4 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'D{column}')
    transfer1 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate1.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))        
    transfer2 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate2.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))
    transfer3 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate3.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))
    transfer4 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate4.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))
    
# Plate 6hr timepoint samples
for i_sample, sample in enumerate(timepoint_samples_6hrs):

    # Plate one sample per column, replicates in rows E-H
    column = i_sample + 1
    replicate1 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'E{column}')
    replicate2 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'F{column}')
    replicate3 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'G{column}')
    replicate4 = protocol.primitive_step('PlateCoordinates', 
                                        source=plate.output_pin('samples'),
                                        coordinates=f'H{column}')
    transfer1 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate1.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))        
    transfer2 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate2.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))
    transfer3 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate3.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))
    transfer4 = protocol.primitive_step('Transfer',
                                   source=sample.output_pin('samples'),
                                   destination=replicate4.output_pin('samples'),
                                   amount=sbol3.Measure(100, OM.microliter))
    
# Plate blanks in column 9
for row in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    blank_wells = protocol.primitive_step('PlateCoordinates', 
                                          source=plate.output_pin('samples'),
                                          coordinates=f'{row}9')
    transfer = protocol.primitive_step('Transfer',
                                       source=lb_cam,
                                       destination=blank_wells.output_pin('samples'),
                                       amount=sbol3.Measure(100, OM.microliter))

# Perform measurements
scan_coordinates = protocol.primitive_step('PlateCoordinates', 
                                           source=plate.output_pin('samples'),
                                           coordinates=f'A1:H9')
measure_absorbance = protocol.primitive_step('MeasureAbsorbance',
                                             samples=scan_coordinates.output_pin('samples'),
                                             wavelength=sbol3.Measure(600, OM.nanometer))
measure_fluorescence = protocol.primitive_step('MeasureFluorescence',
                                               samples=scan_coordinates.output_pin('samples'),
                                               excitationWavelength=sbol3.Measure(485, OM.nanometer),
                                               emissionWavelength=sbol3.Measure(530, OM.nanometer),
                                               emissionBandpassWidth=sbol3.Measure(30, OM.nanometer))

protocol.designate_output('measurements', 'http://bioprotocols.org/paml#SampleData', source=measure_absorbance.output_pin('measurements'))
protocol.designate_output('measurements', 'http://bioprotocols.org/paml#SampleData', source=measure_aborbance.output_pin('measurements'))
protocol.designate_output('measurements', 'http://bioprotocols.org/paml#SampleData', source=measure_absorbance.output_pin('measurements'))

In [ ]:
# Execute the protocol

from IPython.display import Markdown

agent = sbol3.Agent("test_agent")
ee = ExecutionEngine(specializations=[MarkdownSpecialization("test_LUDOX_markdown.md")])
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=[])
Markdown(ee.specializations[0].markdown)


In [ ]:
print(ee.specializations[0].markdown)

In [ ]:
text_file = open('interlab-growth-curve.md', "w")
n = text_file.write(ee.specializations[0].markdown)
text_file.close()